In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 200     # number of variables
num_ineq = 200    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msLinear
model = msLinear(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    try:
        xval, objval = model.solve("gurobi")
        # eval
        params.append(list(b.cpu().numpy()))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(b.cpu().numpy()))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/ln_exact_200-200.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:40:48<00:00, 60.48s/it]


       Elapsed Time
count    100.000000
mean      60.481189
std        0.032173
min       60.412278
25%       60.458226
50%       60.482873
75%       60.505656
max       60.573851
Number of infeasible solution: 0
Number of None values:  100


## Heuristic - Round

In [18]:
from src.heuristic import naive_round

In [19]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    try:
        xval_rel, _ = model_rel.solve("gurobi")
        xval, objval = naive_round(xval_rel, model)
        # eval
        params.append(list(b.cpu().numpy()))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(b.cpu().numpy()))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/ln_heur_rnd_200-200.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:05<00:00,  1.26s/it]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean  -172.915954         11.712249      0.414948
std      0.441110          1.520185      0.073842
min   -173.875061          7.938176      0.355120
25%   -173.234474         10.614469      0.368889
50%   -172.926655         11.520263      0.376018
75%   -172.585474         12.504157      0.430602
max   -171.680405         16.651590      0.660999
Number of infeasible solution: 100
Number of None values:  0


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    try:
        xval, objval = model_heur.solve("gurobi")
        # eval
        params.append(list(b))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model_heur.cal_violation()))
    except:
        params.append(list(b))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/ln_heur_n1_200-200.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [15:37<00:00,  9.38s/it]


ValueError: All arrays must be of the same length

## Learnable Rounding

In [20]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [21]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [22]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [23]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 9255.82
Epoch 1, Validation Loss: -21.64
Epoch 2, Validation Loss: -36.63
Epoch 3, Validation Loss: -57.95
Epoch 4, Validation Loss: -78.46
Epoch 5, Validation Loss: -82.78
Epoch 6, Validation Loss: -80.96
Epoch 7, Validation Loss: -97.18
Epoch 8, Validation Loss: -46.63
Epoch 9, Validation Loss: -73.47
Epoch 10, Validation Loss: -87.08
Epoch 11, Validation Loss: -48.10
Epoch 12, Validation Loss: -107.01
Epoch 13, Validation Loss: -111.50
Epoch 14, Validation Loss: -112.37
Epoch 15, Validation Loss: -110.54
Epoch 16, Validation Loss: -125.96
Epoch 17, Validation Loss: -89.43
Epoch 18, Validation Loss: -132.55
Epoch 19, Validation Loss: -132.82
Epoch 20, Validation Loss: -76.69
Epoch 21, Validation Loss: -123.48
Epoch 22, Validation Loss: -138.29
Epoch 23, Validation Loss: -129.41
Epoch 24, Validation Loss: -137.11
Epoch 25, Validation Loss: -133.70
Epoch 26, Validation Loss: -139.79
Epoch 27, Validation Loss: -139.31
Epoch 28, Validation Loss: -142.44
Epoch 29

In [24]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lr_200-200.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.29it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean  -143.271993          0.004226      0.003413
std      1.381116          0.019353      0.000813
min   -145.451933          0.000000      0.001967
25%   -144.158258          0.000000      0.003001
50%   -143.774509          0.000000      0.003162
75%   -142.548244          0.000000      0.003769
max   -139.070834          0.138614      0.006208
Number of infeasible solution: 6


## Learnable Threshold

In [25]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [26]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [27]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [28]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 5004.20
Epoch 1, Validation Loss: -45.97
Epoch 2, Validation Loss: -19.76
Epoch 3, Validation Loss: -26.31
Epoch 4, Validation Loss: -79.59
Epoch 5, Validation Loss: -74.84
Epoch 6, Validation Loss: -82.11
Epoch 7, Validation Loss: -96.68
Epoch 8, Validation Loss: -101.14
Epoch 9, Validation Loss: 73.48
Epoch 10, Validation Loss: -89.17
Epoch 11, Validation Loss: -100.19
Epoch 12, Validation Loss: -94.68
Epoch 13, Validation Loss: -93.51
Epoch 14, Validation Loss: -107.89
Epoch 15, Validation Loss: -119.03
Epoch 16, Validation Loss: -122.29
Epoch 17, Validation Loss: -132.14
Epoch 18, Validation Loss: -128.82
Epoch 19, Validation Loss: -90.11
Epoch 20, Validation Loss: -105.28
Epoch 21, Validation Loss: -127.07
Epoch 22, Validation Loss: -88.65
Epoch 23, Validation Loss: -34.92
Epoch 24, Validation Loss: -138.54
Epoch 25, Validation Loss: -126.90
Epoch 26, Validation Loss: -122.93
Epoch 27, Validation Loss: -139.87
Epoch 28, Validation Loss: -142.24
Epoch 29, 

In [29]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lt_200-200.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.21it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean  -150.722680          0.001161      0.003838
std      0.118693          0.009434      0.001032
min   -150.940154          0.000000      0.001996
25%   -150.802877          0.000000      0.003001
50%   -150.738322          0.000000      0.003521
75%   -150.704321          0.000000      0.004512
max   -150.056339          0.092000      0.007387
Number of infeasible solution: 3


### Parametric Learning Then Rounding

In [30]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [31]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [32]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmLinear(["b", "x"], num_var, num_ineq, penalty_weight)

In [33]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 5051.06
Epoch 1, Validation Loss: -36.89
Epoch 2, Validation Loss: 7.74
Epoch 3, Validation Loss: -63.83
Epoch 4, Validation Loss: -80.71
Epoch 5, Validation Loss: -81.09
Epoch 6, Validation Loss: -63.37
Epoch 7, Validation Loss: -82.46
Epoch 8, Validation Loss: -96.28
Epoch 9, Validation Loss: -99.80
Epoch 10, Validation Loss: -96.16
Epoch 11, Validation Loss: -103.05
Epoch 12, Validation Loss: -110.83
Epoch 13, Validation Loss: -108.77
Epoch 14, Validation Loss: -109.20
Epoch 15, Validation Loss: 110.05
Epoch 16, Validation Loss: -105.38
Epoch 17, Validation Loss: -120.44
Epoch 18, Validation Loss: -12.38
Epoch 19, Validation Loss: -114.14
Epoch 20, Validation Loss: -129.72
Epoch 21, Validation Loss: -133.75
Epoch 22, Validation Loss: -131.80
Epoch 23, Validation Loss: -137.62
Epoch 24, Validation Loss: -138.85
Epoch 25, Validation Loss: -139.17
Epoch 26, Validation Loss: -139.47
Epoch 27, Validation Loss: -136.91
Epoch 28, Validation Loss: -141.07
Epoch 29,

In [34]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_pr_200-200.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:11<00:00,  8.77it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean  -154.078087          0.257414      0.000934
std      0.028788          0.247080      0.000525
min   -154.159454          0.000000      0.000000
25%   -154.086661          0.043950      0.000997
50%   -154.060520          0.187798      0.001005
75%   -154.060520          0.436228      0.001099
max   -154.060520          1.102671      0.002430
Number of infeasible solution: 81
